In [ ]:
import torch
import torch.utils.data as data
import numpy as np

In [ ]:
class DataLoader(data.Dataset):

    def __init__(self) -> None:
        super().__init__()

    def __getitem__(self, index):
        image_id = 0
        image = np.random.randn(512)
        encoded_input_id = torch.tensor(list(range(self.max_input_len)))
        encoded_input_attention_mask = torch.tensor([0]*self.max_input_len)
        encoded_question_id = torch.tensor(list(range(self.max_q_len)))
        encoded_question_attention_mask = torch.tensor([0]*self.max_q_len)
        encoded_cat_obj_ids = torch.tensor(list(range(self.max_inference_len)))
        encoded_cat_obj_attn_mask = torch.tensor([0]*self.max_inference_len)
        rcnn_features = torch.tensor(0)
        rcnn_locations = torch.tensor(0)

        return image_id, torch.from_numpy(image), encoded_input_id, encoded_input_attention_mask, encoded_question_id, encoded_question_attention_mask, encoded_cat_obj_ids, encoded_cat_obj_attn_mask, rcnn_features, rcnn_locations

    def __len__(self):
        return 300


In [ ]:
def collate_fn(data):
    image_ids, images, encoded_input_ids, encoded_input_attention_masks, encoded_question_ids, encoded_question_attention_masks, encoded_cat_obj_ids, encoded_cat_obj_attn_mask, rcnn_features, rcnn_locations = list(zip(*data))

    images = torch.stack(images).float()
    input_ids = torch.stack(encoded_input_ids).long()
    input_attention_masks = torch.stack(encoded_input_attention_masks).long()
    question_ids = torch.stack(encoded_question_ids).long()
    question_attention_masks = torch.stack(encoded_question_attention_masks).long()
    inference_ids = torch.stack(encoded_cat_obj_ids).long()
    inference_attention_masks = torch.stack(encoded_cat_obj_attn_mask).long()
    rcnn_features = torch.stack(rcnn_features)
    rcnn_locations = torch.stack(rcnn_locations)

    return {"images": images,
            "image_ids": image_ids,
            "question_ids": question_ids,
            "question_attention_masks": question_attention_masks,
            "input_ids": input_ids,
            "input_attention_masks": input_attention_masks,
            "inference_ids": inference_ids,
            "inference_attention_masks": inference_attention_masks,
            "rcnn_features": rcnn_features,
            "rcnn_locations": rcnn_locations}

In [ ]:
def get_loader(dataset, tokenizer, batch_size, sampler=None,
               shuffle=True, num_workers=1, max_examples=None,
               indices=None):

    vqg = VQGDataset(dataset, tokenizer, max_examples=max_examples,
                     indices=indices)
    data_loader = torch.utils.data.DataLoader(dataset=vqg,
                                              batch_size=batch_size,
                                              shuffle=shuffle,
                                              sampler=sampler,
                                              num_workers=num_workers,
                                              collate_fn=collate_fn)
    return data_loader


In [ ]:
class Model(nn.Module):